In [1]:
import time
import numpy as np
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)


    # Handles do robô e rodas
    #robotHandle = sim.getObject('Pioneer_p3dx') # MAC
    #l_wheel = sim.getObject('Pioneer_p3dx_leftMotor') # MAC
    #r_wheel = sim.getObject('Pioneer_p3dx_rightMotor') # MAC
    
    robotHandle = sim.getObject('/PioneerP3DX') # Linux
    l_wheel = sim.getObject('/PioneerP3DX/leftMotor') # Linux
    r_wheel = sim.getObject('/PioneerP3DX/rightMotor') # Linux


    # Handles dos sensores ultra-sônicos
    #sensorHandles = [sim.getObject(f'Pioneer_p3dx_ultrasonicSensor{i}') for i in range(1, 17)] # MAC
    sensorHandles = [sim.getObject(f'/PioneerP3DX/ultrasonicSensor[{i}]') for i in range(16)] # Linux

    # Constantes do robô
    L = 0.331
    r = 0.09751

    # Ganhos do controlador
    kv = 1.5
    kw = 3.5
    K_att = 1.0
    K_rep = 0.5

    # Posição do objetivo (em metros)
    x_goal = np.array([2.0, 2.0])  # altere para a posição desejada

    # Estado inicial
    q = np.array([0.0, 0.0, 0.0])  # x, y, theta

    # Iniciar simulação
    if sim.getSimulationState() != 0:
        sim.stopSimulation()
        time.sleep(1)
    sim.startSimulation()
    sim.step()

    print("Iniciando controle do robô...")
    while np.linalg.norm(x_goal - q[:2]) > 0.5:  # enquanto não estiver a 50 cm do goal
        sim_time = sim.getSimulationTime()
        dt = sim.getSimulationTimeStep()

        ### 1. FORÇA DE REPULSÃO ###
        F_rep = np.zeros(2)
        for sensor in sensorHandles:
            res = sim.readProximitySensor(sensor)
            result, distance, detectedPoint = res[0], res[1], res[2]
            if result:  # obstáculo detectado
                detectedPoint = np.array(detectedPoint[:3])
                dist = np.linalg.norm(distance)
                if dist < 0.5:  # considerar apenas obstáculos próximos
                    direction = -detectedPoint[:2] / dist  # vetor para longe do obstáculo (2D)
                    F_rep += K_rep * direction * (1.0 / dist**2)  # força repulsiva proporcional

        ### 2. FORÇA DE ATRAÇÃO ###
        F_att = K_att * (x_goal - q[:2])

        ### 3. FORÇA RESULTANTE ###
        F = F_att + F_rep  # atração + repulsão (como na imagem 1)

        ### 4. CONTROLE CINEMÁTICO COM PSEUDO-INVERSA ###
        Fx, Fy = F
        theta = q[2]
        v = kv * (Fx * np.cos(theta) + Fy * np.sin(theta))
        w = kw * (np.arctan2(Fy, Fx) - theta)

        ### 5. CINEMÁTICA INVERSA PARA VELOCIDADES DAS RODAS ###
        wr = ((2.0 * v) + (w * L)) / (2.0 * r)
        wl = ((2.0 * v) - (w * L)) / (2.0 * r)

        # Enviar comandos
        sim.setJointTargetVelocity(l_wheel, wl)
        sim.setJointTargetVelocity(r_wheel, wr)

        ### 6. CINEMÁTICA DIRETA PARA ATUALIZAR POSIÇÃO ESTIMADA ###
        Mdir = np.array([
            [r * np.cos(q[2]) / 2, r * np.cos(q[2]) / 2],
            [r * np.sin(q[2]) / 2, r * np.sin(q[2]) / 2],
            [r / L, -r / L]
        ])
        u = np.array([wr, wl])
        q = q + (Mdir @ u) * dt

        print(f"[{sim_time:.2f}s] Pose: {q[:2]} θ={np.rad2deg(q[2]):.1f}°  | Dist Goal: {np.linalg.norm(x_goal - q[:2]):.2f} m")

        sim.step()

    ### FIM DA SIMULAÇÃO ###
    print("Parando robô...")
    sim.setJointTargetVelocity(l_wheel, 0)
    sim.setJointTargetVelocity(r_wheel, 0)

    print("Fim da simulação.")
    print('Posição estimada final:', q[:2], 'θ=', np.rad2deg(q[2]))

    sim.stopSimulation()

except Exception as e:
    print(f"Ocorreu um erro: {e}")

print("Programa finalizado.")


Iniciando controle do robô...
[0.10s] Pose: [0.15 0.  ] θ=7.9°  | Dist Goal: 2.72 m
[0.40s] Pose: [0.3073323  0.02176169] θ=15.2°  | Dist Goal: 2.60 m
[0.60s] Pose: [0.46414248 0.06428391] θ=21.8°  | Dist Goal: 2.47 m
[0.85s] Pose: [0.61810192 0.12571448] θ=27.6°  | Dist Goal: 2.33 m
[1.20s] Pose: [0.76532783 0.20283316] θ=33.0°  | Dist Goal: 2.18 m
[1.45s] Pose: [0.90384948 0.29283118] θ=38.0°  | Dist Goal: 2.03 m
[1.75s] Pose: [1.03052409 0.39187592] θ=42.6°  | Dist Goal: 1.88 m
[2.05s] Pose: [1.14380308 0.4959597 ] θ=46.8°  | Dist Goal: 1.73 m
[2.40s] Pose: [1.24229506 0.60100716] θ=51.0°  | Dist Goal: 1.59 m
[2.60s] Pose: [1.3279096  0.70658803] θ=54.8°  | Dist Goal: 1.46 m
[2.90s] Pose: [1.40125489 0.81056238] θ=58.7°  | Dist Goal: 1.33 m
[3.05s] Pose: [1.47059354 0.92440778] θ=59.9°  | Dist Goal: 1.20 m
[3.30s] Pose: [1.53210297 1.03059462] θ=61.3°  | Dist Goal: 1.08 m
[3.55s] Pose: [1.58616156 1.12917483] θ=62.6°  | Dist Goal: 0.96 m
[3.70s] Pose: [1.63363769 1.22075352] θ=64.1°